In [1]:
import pyspark
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession


In [2]:
spark = SparkSession.builder.appName('test').master("spark://bphani.local:7077").getOrCreate()


In [3]:
df = spark.read.csv('/Users/phaneendra/Downloads/kill_match_stats_final_0.csv', inferSchema=True, header=True)

In [4]:
df.schema

StructType(List(StructField(killed_by,StringType,true),StructField(killer_name,StringType,true),StructField(killer_placement,DoubleType,true),StructField(killer_position_x,DoubleType,true),StructField(killer_position_y,DoubleType,true),StructField(map,StringType,true),StructField(match_id,StringType,true),StructField(time,IntegerType,true),StructField(victim_name,StringType,true),StructField(victim_placement,DoubleType,true),StructField(victim_position_x,DoubleType,true),StructField(victim_position_y,DoubleType,true)))

In [5]:
df.show()

+------------+----------------+----------------+-----------------+-----------------+-------+--------------------+----+---------------+----------------+-----------------+-----------------+
|   killed_by|     killer_name|killer_placement|killer_position_x|killer_position_y|    map|            match_id|time|    victim_name|victim_placement|victim_position_x|victim_position_y|
+------------+----------------+----------------+-----------------+-----------------+-------+--------------------+----+---------------+----------------+-----------------+-----------------+
|     Grenade| KrazyPortuguese|             5.0|         657725.1|         146275.2|MIRAMAR|2U4GBNA0YmnLSqvEy...| 823|KrazyPortuguese|             5.0|         657725.1|         146275.2|
|      SCAR-L|nide2Bxiaojiejie|            31.0|         93091.37|         722236.4|MIRAMAR|2U4GBNA0YmnLSqvEy...| 194|    X3evolution|            33.0|         92238.68|         723375.1|
|        S686|        Ascholes|            43.0|         366

In [8]:
key = "map"
df1 = df.select(key).groupBy(key).count().sort(desc("count"))
rlis = df1.limit(5).select(key).collect()
rlis
total = df1.count()

In [9]:

input_columns = ["killed_by", "map"]
df.select(input_columns).schema

StructType(List(StructField(killed_by,StringType,true),StructField(map,StringType,true)))

In [6]:
from pyspark.ml.feature import OneHotEncoder
from pyspark.ml.feature import StringIndexer, StringIndexerModel
from pyspark.sql.functions import col, when,desc
import pyspark.sql.functions as F
from pyspark.sql.functions import udf

cap = 5



input_columns = ["killed_by", "map"]
idx_cols = [x+'_idx' for x in input_columns]
vec_cols = [y+'_vec' for y in input_columns]


    
input_arrays = {}

for key in input_columns:
    print(f"Key: {key}")
    key_df = df.select(key).groupBy(key).count().sort(desc("count"))
    row_list = key_df.limit(cap).select(key).collect()
    total_count = key_df.count()
    _array = [row[0] for row in row_list]
    if total_count>cap:
        _array.append("OTHER")
    print(f"length:{len(_array)}")
    input_arrays[key] = _array 



Key: killed_by
length:6
Key: map
length:3


In [7]:
# Indexer

def index_cols(row):
    print(row)
    new_val_list = []
    for z in input_columns:
        val = float(len(input_arrays[z]))
        if row[z] in input_arrays[z]:
            val = float(input_arrays[z].index(row[z]))
        new_val_list.append(val)
        
    x = tuple(map(lambda x: row[x], row.__fields__)) + tuple(new_val_list)
    print(x)
    return x
    


new_idx_cols = df.columns + idx_cols
newdf = df.rdd.map(index_cols).toDF(new_idx_cols)

newdf.select(input_columns + idx_cols).show(3)

+---------+-------+-------------+-------+
|killed_by|    map|killed_by_idx|map_idx|
+---------+-------+-------------+-------+
|  Grenade|MIRAMAR|          6.0|    1.0|
|   SCAR-L|MIRAMAR|          2.0|    1.0|
|     S686|MIRAMAR|          6.0|    1.0|
+---------+-------+-------------+-------+
only showing top 3 rows



In [16]:
input_arrays

{'killed_by': ['Down and Out', 'M416', 'SCAR-L', 'M16A4', 'AKM'],
 'map': ['ERANGEL', 'MIRAMAR', None]}

['killed_by_Down and Out',
 'killed_by_M416',
 'killed_by_SCAR-L',
 'killed_by_M16A4',
 'killed_by_AKM',
 'killed_by_OTHER',
 'map_ERANGEL',
 'map_MIRAMAR',
 'map_None',
 'map_OTHER']

In [6]:
# Custom onehot



def index_cols(row):
    print(row)
    new_val_list = []
    for z in input_columns:
        res = []
        for val in range(0, len(input_arrays[z]) - 1):
            res.append(0)
        index = len(input_arrays[z])-1
        if row[z] in input_arrays[z]:
            index = input_arrays[z].index(row[z])
        res.insert(index, 1)
        new_val_list.extend(res)
        
    x = tuple(map(lambda x: row[x], row.__fields__)) + tuple(new_val_list)
    print(x)
    return x
    

new_oh_cols = []
for z in input_columns:
    new_oh_cols.extend([z +'_'+ str(ohc) for ohc in input_arrays[z]])


new_idx_cols = df.columns + new_oh_cols
newdf = df.rdd.map(index_cols).toDF(new_idx_cols)

newdf.select(input_columns + new_oh_cols).show(3)

+---------+-------+----------------------+--------------+----------------+---------------+-------------+---------------+-----------+-----------+--------+
|killed_by|    map|killed_by_Down and Out|killed_by_M416|killed_by_SCAR-L|killed_by_M16A4|killed_by_AKM|killed_by_OTHER|map_ERANGEL|map_MIRAMAR|map_None|
+---------+-------+----------------------+--------------+----------------+---------------+-------------+---------------+-----------+-----------+--------+
|  Grenade|MIRAMAR|                     0|             0|               0|              0|            0|              1|          0|          1|       0|
|   SCAR-L|MIRAMAR|                     0|             0|               1|              0|            0|              0|          0|          1|       0|
|     S686|MIRAMAR|                     0|             0|               0|              0|            0|              1|          0|          1|       0|
+---------+-------+----------------------+--------------+----------------+--

In [ ]:

# "killed_by", "map", "killer_name"   "killed_by_idx", "map_idx", "killer_name_idx" 
 
# indexed = StringIndexerModel.from_arrays_of_labels(input_arrays,inputCols=input_columns,outputCols=idx_cols,handleInvalid='keep').transform(df)
# str_indexer = StringIndexer(inputCols=input_columns, outputCols=idx_cols, handleInvalid="keep")
# indexer = str_indexer.fit(df)
# indexed = indexer.transform(df)
indexed.select(idx_cols).show()

In [8]:
one_encoder = OneHotEncoder(inputCols=["killed_by_idx","map_idx"], outputCols=["killed_by_vec","map_vec"], dropLast=False, handleInvalid="keep")
model = one_encoder.fit(newdf)
encoded = model.transform(newdf)
encoded.show()

+------------+----------------+----------------+-----------------+-----------------+-------+--------------------+----+---------------+----------------+-----------------+-----------------+-------------+-------+-------------+-------------+
|   killed_by|     killer_name|killer_placement|killer_position_x|killer_position_y|    map|            match_id|time|    victim_name|victim_placement|victim_position_x|victim_position_y|killed_by_idx|map_idx|killed_by_vec|      map_vec|
+------------+----------------+----------------+-----------------+-----------------+-------+--------------------+----+---------------+----------------+-----------------+-----------------+-------------+-------+-------------+-------------+
|     Grenade| KrazyPortuguese|             5.0|         657725.1|         146275.2|MIRAMAR|2U4GBNA0YmnLSqvEy...| 823|KrazyPortuguese|             5.0|         657725.1|         146275.2|          6.0|    1.0|(8,[6],[1.0])|(4,[1],[1.0])|
|      SCAR-L|nide2Bxiaojiejie|            31.0|

In [8]:
encoded.columns

['killed_by',
 'killer_name',
 'killer_placement',
 'killer_position_x',
 'killer_position_y',
 'map',
 'match_id',
 'time',
 'victim_name',
 'victim_placement',
 'victim_position_x',
 'victim_position_y',
 'killed_by_idx',
 'map_idx',
 'killed_by_vec',
 'map_vec']

In [10]:


colIdx = encoded.select(["map","map_idx"]).distinct().rdd.collectAsMap()
print(colIdx)
colIdx =  sorted((value, "ls_" + str(key)) for (key, value) in colIdx.items())

newCols = list(map(lambda x: x[1], colIdx))
actualCol = encoded.columns
allColNames = actualCol + newCols


{'ERANGEL': 0.0, None: 2.0, 'MIRAMAR': 1.0}


In [61]:
for x,y in colIdx:
    print(int(x),y)

0 ls_ERANGEL
1 ls_MIRAMAR
2 ls_None


In [62]:
# from pyspark.sql.types import DoubleType
# from pyspark.sql.functions import lit, udf

# def ith_(v, i):
#     try:
#         return float(v[i])
#     except ValueError:
#         return None

# ith = udf(ith_, DoubleType())


# for x,y in colIdx:
#     print(int(x),y)
#     encoded.withColumn(y, ith("map_vec", lit(int(x))))

# encoded.show()


In [9]:
def extract(row):
    print(row)
    x = tuple(map(lambda x: row[x], row.__fields__)) + tuple(row.map_vec.toArray().tolist())
    print(x)
    return x

result = encoded.rdd.map(extract).toDF()
result.show()

+------------+----------------+----+--------+--------+-------+--------------------+----+---------------+----+--------+--------+---+---+-------------+-------------+---+---+---+---+
|          _1|              _2|  _3|      _4|      _5|     _6|                  _7|  _8|             _9| _10|     _11|     _12|_13|_14|          _15|          _16|_17|_18|_19|_20|
+------------+----------------+----+--------+--------+-------+--------------------+----+---------------+----+--------+--------+---+---+-------------+-------------+---+---+---+---+
|     Grenade| KrazyPortuguese| 5.0|657725.1|146275.2|MIRAMAR|2U4GBNA0YmnLSqvEy...| 823|KrazyPortuguese| 5.0|657725.1|146275.2|5.0|1.0|(7,[5],[1.0])|(4,[1],[1.0])|0.0|1.0|0.0|0.0|
|      SCAR-L|nide2Bxiaojiejie|31.0|93091.37|722236.4|MIRAMAR|2U4GBNA0YmnLSqvEy...| 194|    X3evolution|33.0|92238.68|723375.1|2.0|1.0|(7,[2],[1.0])|(4,[1],[1.0])|0.0|1.0|0.0|0.0|
|        S686|        Ascholes|43.0|366921.4|421623.9|MIRAMAR|2U4GBNA0YmnLSqvEy...| 103|        Ctrl

In [11]:
result2 = result.toDF(*allColNames)


IllegalArgumentException: requirement failed: The number of columns doesn't match.
Old column names (20): _1, _2, _3, _4, _5, _6, _7, _8, _9, _10, _11, _12, _13, _14, _15, _16, _17, _18, _19, _20
New column names (19): killed_by, killer_name, killer_placement, killer_position_x, killer_position_y, map, match_id, time, victim_name, victim_placement, victim_position_x, victim_position_y, killed_by_idx, map_idx, killed_by_vec, map_vec, ls_ERANGEL, ls_MIRAMAR, ls_None

In [ ]:
result2.show()